In [1]:
import os
os.environ['CUDA_MODULE_LOADING'] = 'LAZY'

from ultralytics import YOLO
import torch
import yaml
from pathlib import Path

# Kiểm tra PyTorch và CUDA
print(f"✅ PyTorch version: {torch.__version__}")
print(f"✅ CUDA available: {torch.cuda.is_available()}")

if torch.cuda.is_available():
    print(f"✅ Number of CUDA devices: {torch.cuda.device_count()}")
    print(f"✅ CUDA device 0: {torch.cuda.get_device_name(0)}")
    
    total_memory = torch.cuda.get_device_properties(0).total_memory / 1e9
    print(f"✅ Total GPU memory: {total_memory:.2f} GB")
    
    test_tensor = torch.zeros(1).cuda()
    print(f"✅ CUDA test successful!")
    del test_tensor
    torch.cuda.empty_cache()

device = 'cuda:0' if torch.cuda.is_available() else 'cpu'
print(f"\n🎯 Using device: {device}")

✅ PyTorch version: 2.6.0+cu124
✅ CUDA available: True
✅ Number of CUDA devices: 1
✅ CUDA device 0: NVIDIA GeForce RTX 3060 Laptop GPU
✅ Total GPU memory: 6.44 GB
✅ CUDA test successful!

🎯 Using device: cuda:0


In [2]:
# Verify Dataset YAML 
dataset_yaml = Path('F:\pr\data\dataset.yaml')

with open(dataset_yaml, 'r') as f:
    config = yaml.safe_load(f)
    
print("📋 Dataset Configuration:")
print(yaml.dump(config, default_flow_style=False, allow_unicode=True))

📋 Dataset Configuration:
names:
- BSD
- BSV
nc: 2
path: F:\pr\data
test: images/test/
train: images/train/
val: images/val/



<>:2: SyntaxWarning: invalid escape sequence '\p'
<>:2: SyntaxWarning: invalid escape sequence '\p'
C:\Users\Legion\AppData\Local\Temp\ipykernel_9052\736586091.py:2: SyntaxWarning: invalid escape sequence '\p'
  dataset_yaml = Path('F:\pr\data\dataset.yaml')


In [3]:
# Initialize YOLO Model 
model_size = 'yolov8n' 

print(f"🚀 Loading YOLOv8 {model_size} model...")
model = YOLO(f'{model_size}.pt')  

print("✅ Model loaded successfully!")
print(f"Model summary:")
model.info()

🚀 Loading YOLOv8 yolov8n model...
✅ Model loaded successfully!
Model summary:
YOLOv8n summary: 129 layers, 3,157,200 parameters, 0 gradients, 8.9 GFLOPs


(129, 3157200, 0, 8.8575488)

In [4]:
# Configure Training Parameters
train_config = {
    'data': str(dataset_yaml),
    'epochs': 100,
    'imgsz': 640,
    'batch': 16,  # Điều chỉnh tùy GPU memory
    'device': 0 if torch.cuda.is_available() else 'cpu',
    'workers': 4,
    'project': 'runs/detect',
    'name': 'license_plate_v1',
    'exist_ok': False,
    'pretrained': True,
    'optimizer': 'Adam',
    'lr0': 0.001,
    'patience': 20,  # Early stopping
    'save': True,
    'save_period': 10,  # Save checkpoint every 10 epochs
    'cache': False,  # Set True if enough RAM
    'amp': True,  # Automatic Mixed Precision
    'verbose': True,
    
    # Data Augmentation
    'hsv_h': 0.015,
    'hsv_s': 0.7,
    'hsv_v': 0.4,
    'degrees': 10.0,
    'translate': 0.1,
    'scale': 0.5,
    'shear': 0.0,
    'perspective': 0.0,
    'flipud': 0.0,
    'fliplr': 0.5,
    'mosaic': 1.0,
    'mixup': 0.0,
}

print("⚙️ Training Configuration:")
for key, value in train_config.items():
    print(f"  {key}: {value}")

⚙️ Training Configuration:
  data: F:\pr\data\dataset.yaml
  epochs: 100
  imgsz: 640
  batch: 16
  device: 0
  workers: 4
  project: runs/detect
  name: license_plate_v1
  exist_ok: False
  pretrained: True
  optimizer: Adam
  lr0: 0.001
  patience: 20
  save: True
  save_period: 10
  cache: False
  amp: True
  verbose: True
  hsv_h: 0.015
  hsv_s: 0.7
  hsv_v: 0.4
  degrees: 10.0
  translate: 0.1
  scale: 0.5
  shear: 0.0
  perspective: 0.0
  flipud: 0.0
  fliplr: 0.5
  mosaic: 1.0
  mixup: 0.0


In [5]:
# Start Training 
print("\n🎯 Starting training...")
print("="*50)

results = model.train(**train_config)

print("\n✅ Training completed!")


🎯 Starting training...
New https://pypi.org/project/ultralytics/8.3.240 available  Update with 'pip install -U ultralytics'
Ultralytics 8.3.233  Python-3.13.9 torch-2.6.0+cu124 CUDA:0 (NVIDIA GeForce RTX 3060 Laptop GPU, 6144MiB)
engine\trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=F:\pr\data\dataset.yaml, degrees=10.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=100, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.001, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, 

In [6]:
# Evaluate Model 
print("📊 Evaluating model on validation set...")
metrics = model.val()

print("\n📈 Validation Metrics:")
print(f"  mAP50: {metrics.box.map50:.4f}")
print(f"  mAP50-95: {metrics.box.map:.4f}")
print(f"  Precision: {metrics.box.mp:.4f}")
print(f"  Recall: {metrics.box.mr:.4f}")

📊 Evaluating model on validation set...
Ultralytics 8.3.233  Python-3.13.9 torch-2.6.0+cu124 CUDA:0 (NVIDIA GeForce RTX 3060 Laptop GPU, 6144MiB)
Model summary (fused): 72 layers, 3,006,038 parameters, 0 gradients, 8.1 GFLOPs
val: Fast image access  (ping: 0.10.1 ms, read: 22.35.9 MB/s, size: 159.7 KB)
val: Scanning F:\pr\data\labels\val.cache... 1145 images, 0 backgrounds, 0 corrupt: 100% ━━━━━━━━━━━━ 1145/1145 1.1Mit/s 0.0ss
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 72/72 11.7it/s 6.2s0.2s
                   all       1145       1313      0.986      0.989      0.994      0.915
                   BSD        409        410      0.993      0.995      0.995      0.909
                   BSV        753        903      0.979      0.984      0.994       0.92
Speed: 0.8ms preprocess, 2.3ms inference, 0.0ms loss, 0.6ms postprocess per image
Results saved to F:\pr\runs\detect\val2

📈 Validation Metrics:
  mAP50: 0.9944
  mAP50-9

In [7]:
# Visualize Training Results
import matplotlib.pyplot as plt
from PIL import Image

# Path to results
results_dir = Path('F:/pr/runs/detect/license_plate_v12')

# Load and display training curves
results_img = Image.open(results_dir / 'results.png')
plt.figure(figsize=(16, 10))
plt.imshow(results_img)
plt.axis('off')
plt.title('Training Results', fontsize=16, fontweight='bold')
plt.tight_layout()
plt.show()

# Confusion matrix
if (results_dir / 'confusion_matrix.png').exists():
    conf_matrix = Image.open(results_dir / 'confusion_matrix.png')
    plt.figure(figsize=(10, 8))
    plt.imshow(conf_matrix)
    plt.axis('off')
    plt.title('Confusion Matrix', fontsize=16, fontweight='bold')
    plt.tight_layout()
    plt.show()

<Figure size 1600x1000 with 1 Axes>

<Figure size 1000x800 with 1 Axes>

In [8]:
# Test Predictions on Sample Images
# Load best model
best_model = YOLO(results_dir / 'weights' / 'best.pt')

# Predict on validation images
val_images = list(Path('images/val').glob('*.jpg'))[:9]

fig, axes = plt.subplots(3, 3, figsize=(15, 15))
axes = axes.flatten()

for idx, img_path in enumerate(val_images):
    # Predict
    results = best_model.predict(source=str(img_path), conf=0.25, iou=0.45)
    
    # Get annotated image
    annotated = results[0].plot()
    
    # Convert BGR to RGB
    annotated_rgb = cv2.cvtColor(annotated, cv2.COLOR_BGR2RGB)
    
    axes[idx].imshow(annotated_rgb)
    axes[idx].set_title(f"{img_path.name}", fontsize=10)
    axes[idx].axis('off')

plt.tight_layout()
plt.savefig('predictions_sample.png', dpi=150, bbox_inches='tight')
plt.show()
print("✅ Predictions saved as 'predictions_sample.png'")

<Figure size 1500x1500 with 9 Axes>

✅ Predictions saved as 'predictions_sample.png'
